In [1]:
import importlib
from igraph import *
import numpy as np
import time
from numba import njit
from scipy.sparse import coo_matrix

In [2]:
TriangleData=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]

inflation_order=2
card=2
TriangleGraph=Graph.Formula("X->A,Y->A:B,Z->B:C,X->C")
EvansGraph=Graph.Formula("U3->A:C:D,U2->B:C:D,U1->A:B,A->C,B->D")
InstrumentalGraph=Graph.Formula("U1->X->A->B,U2->A:B")
#We are going to need to get some code to work with variable ordering.

In [3]:
try:
    importlib.reload(inflation)
except NameError:
    import InitializeInflation as inflation

In [4]:
start = time.time()
EvansSparseInflationMatrix = inflation.InflationMatrixFromGraph(EvansGraph,2,2)
print('It took', time.time()-start, 'seconds.')
print(EvansSparseInflationMatrix.shape)

['A', 'B', 'C', 'D']
It took 5.44100022315979 seconds.
(136, 322512)


In [5]:
start = time.time()
valid_column_orbits = inflation.ValidColumnOrbitsFromGraph(EvansGraph,2,2)
print('It took', time.time()-start, 'seconds.')
print(valid_column_orbits.shape)

['A', 'B', 'C', 'D']
It took 0.8419997692108154 seconds.
(8, 322512)


In [6]:
start = time.time()
valid_column_orbits = inflation.ValidColumnOrbitsFromGraph(TriangleGraph,2,4)
print('It took', time.time()-start, 'seconds.')
print(valid_column_orbits.shape)

['A', 'B', 'C']
It took 0.8799989223480225 seconds.
(8, 2123776)


In [7]:
TriangleSparseInflationMatrix = inflation.InflationMatrixFromGraph(TriangleGraph,2,4)
b=inflation.FindB(TriangleData,2)
TriangleSparseInflationMatrix.shape

['A', 'B', 'C']


(2080, 2123776)

In [8]:
TriangleSparseInflationMatrix2 = inflation.optimize_inflation_matrix(TriangleSparseInflationMatrix)

In [9]:
Sol,epsilon=inflation.InflationLP(TriangleSparseInflationMatrix,b)

Preprocessing LP for efficiency boost...
Initiating LP
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123777         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 2123777         
  Cones                  : 0               
  Scalar variables       : 2080            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 6               
Optimizer  - solved problem         : the dual        
Optimizer  - Constraints            : 2080
Optimizer  - Cones 

In [14]:
y=np.array(Sol);
y[np.abs(y) < np.max(np.abs(y))/1000] = 0;
y2=y/np.unique(np.abs(y))[1];
y3=np.rint(y2).astype(np.int);
y4=coo_matrix(y3.astype(np.int))
np.unique(y3)

array([-2, -1,  0,  1,  2,  3,  4])

In [15]:
y4

<2080x1 sparse matrix of type '<class 'numpy.int32'>'
	with 430 stored elements in COOrdinate format>

In [16]:
"Validation:";
(y4.T * TriangleSparseInflationMatrix).min()

0

In [17]:
from collections import defaultdict
tally=defaultdict(list)
[tally[item].append(y4.row[i]) for i,item in enumerate(y4.data)]
for coefficient in np.unique(y4.data):
    print({coefficient:np.array(tally[coefficient])})

{-2: array([  47,   59, 1117], dtype=int32)}
{-1: array([ 63, 293], dtype=int32)}
{1: array([  11,   15,   27,   31,   35,   39,  101,  113,  138,  343,  347,
        355,  363,  376,  508,  509,  510,  512,  513,  514,  515,  523,
        563,  564,  566,  567,  568,  569,  570,  617,  618,  619,  620,
        621,  622,  623,  624,  671,  672,  673,  675,  676,  677,  737,
        759,  763,  783,  787,  799,  803,  947, 1105, 1109, 1113, 1275,
       1283, 1309, 1418, 1419, 1420, 1433, 1434, 1435, 1436, 1453, 1454,
       1456, 1469, 1470, 1471, 1472, 1487, 1488, 1489, 1490, 1503, 1504,
       1505, 1506, 1521, 1522, 1523, 1537, 1538, 1539, 1567, 1579, 1736,
       1752, 1895, 1928, 1955, 1986], dtype=int32)}
{2: array([   3,    7,   19,   23,  190,  194,  198,  202,  206,  210,  214,
        218,  222,  226,  230,  234,  238,  242,  246,  253,  257,  265,
        269,  273,  281,  285,  289,  428,  432,  436,  440,  444,  448,
        452,  456,  460,  464,  468,  472,  476,  480, 